In [18]:
import itertools
import math
from contextlib import contextmanager
from dataclasses import dataclass
from functools import partial
from typing import List, Callable, Union, Generator, Literal

import torch
from circuitsvis.attention import attention_pattern
from torch import Tensor
from tqdm.autonotebook import trange, tqdm
from jaxtyping import Float, Int
from transformer_lens import HookedTransformer
from transformer_lens.hook_points import HookPoint

from utils import (
    docstring_metric,
    ioi_metric,
    kl_on_last_token,
    layer_level_connectom,
    sankey_diagram_of_connectome, Connexion,
)
import diego as d

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
model = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
# model = HookedTransformer.from_pretrained("attn-only-4l")

In [21]:
threshold = 0.4
prompt = "When John and Mary went to the store, John gave a book to"
c = d.connectom(model, prompt,
          d.logit_diff_metric(model, ' Mary', ' John'),
          d.ZeroPattern(),
          strategy=partial(d.explore_backtrack, threshold=threshold),
          # strategy=explore_all_pairs,
)
sankey_diagram_of_connectome(model, prompt, c, threshold=threshold)

0it [00:00, ?it/s]

In [12]:
strategy = partial(explore_bisect, threshold=0.5)
connectom(model, prompt,
          LogitDiffMetric(' Mary', ' John', model),
          ZeroPattern(),
          strategy=strategy,
          )

0it [00:00, ?it/s]

[Connexion(source=slice(0, 15, None), target=slice(0, 15, None), score=tensor(-4.6977), note='All layers'),
 Connexion(source=slice(0, 8, None), target=slice(0, 8, None), score=tensor(-6.0758), note='All layers'),
 Connexion(source=slice(8, 15, None), target=slice(8, 15, None), score=tensor(-3.2468), note='All layers'),
 Connexion(source=slice(8, 12, None), target=slice(12, 15, None), score=tensor(-3.1742), note='All layers'),
 Connexion(source=slice(4, 8, None), target=slice(0, 4, None), score=tensor(0.), note='All layers'),
 Connexion(source=slice(12, 15, None), target=slice(8, 12, None), score=tensor(0.), note='All layers'),
 Connexion(source=slice(10, 12, None), target=slice(12, 14, None), score=tensor(0.1059), note='All layers'),
 Connexion(source=slice(8, 10, None), target=slice(14, 15, None), score=tensor(-0.1457), note='All layers'),
 Connexion(source=slice(8, 15, None), target=slice(0, 8, None), score=tensor(0.), note='All layers'),
 Connexion(source=slice(10, 12, None), targe